In [52]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

In [54]:
# Setze die globale Einstellung für die Anzeige von numerischen Werten
pd.set_option('display.float_format', '{:.4f}'.format)

# Pfad zur Excel-Datei
path_to_csv = "../data/trainingsettransformed.csv"

# Lade die Excel-Datei
df = pd.read_csv(path_to_csv)
df

,cTOTEXn,yConnections.N3_log,yConnections.N4_log,yConnections.N5_log,yConnections.N6_log,yConnections.N7_log,yConnections.tot_log,yConnections.streetlights.dir.N7_log,yConnections.streetlights.N7_log,yConnections.streetlights.N6_log,...,rUsageHours.injection.N4_log,rUsageHours.injection.N5,rUsageHours.injection.N6,rUsageHours.injection.N7,rUsageHours.delivered.N3_log,rUsageHours.delivered.N4_log,rUsageHours.delivered.N5_poly,rUsageHours.delivered.N6,rUsageHours.delivered.N7_poly,rUsageHours.delivered.tot_log
0,65381795.5159,3.1355,5.6204,7.7467,9.0990,11.3537,11.4808,9.6269,5.7807,4.0254,...,8.6983,6025.0912,5304.0960,5505.9566,8.7224,8.7053,74531032.3153,5375.9875,101444150.8742,10.1335
1,87582629.7051,3.3322,6.4739,8.2970,9.6817,11.3828,11.5946,6.4708,7.2910,0.0000,...,8.6634,5666.7913,5059.2994,5079.3212,8.9696,8.6666,59646651.6062,5097.3932,50090789.6922,10.1500
2,230256581.3498,4.9273,6.6254,9.2757,11.2516,12.7424,12.9729,0.0000,7.9367,0.0000,...,8.1043,3327.2259,3298.1013,3466.2997,8.4555,8.4563,59284566.1955,4431.3787,128818045.5848,9.8276
3,12975571.1816,0.0000,4.7185,6.4723,7.9906,9.8702,10.0456,8.6392,7.6681,0.0000,...,8.5661,5410.9270,5163.5496,5153.4322,0.0000,8.5679,50747488.3162,5223.6655,80536074.8517,9.5801
4,36097410.9705,2.5649,5.3845,7.0388,9.1369,10.7515,10.9571,0.0000,6.1420,0.0000,...,8.6656,5812.1489,5277.1403,5433.5878,8.6608,8.6711,65671213.2170,5369.6755,81618913.3593,10.0131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,281078503.6421,4.1109,7.1245,9.3260,10.5901,12.9488,13.0660,11.0794,10.2509,0.0000,...,8.6355,5604.0569,5016.2185,5177.0838,8.6849,8.6395,57568321.3259,5090.0238,64221735.6866,9.9986
170,9704108.3940,0.0000,4.1271,6.0259,7.7643,9.5643,9.7454,0.0000,5.1417,0.0000,...,-0.0000,6014.2507,4669.5661,4561.8767,0.0000,0.0000,73380211.8444,4740.6762,22018171.2517,9.6153
171,10670966.3433,0.0000,0.0000,6.8886,8.1068,10.3166,10.4495,0.0000,6.3648,0.0000,...,-0.0000,5321.4421,4653.6730,5001.6144,0.0000,0.0000,49225111.5227,4705.3358,57039920.7557,9.2463
172,38979847.2052,2.8332,5.8201,7.0665,9.5632,10.6866,10.9941,0.0000,5.8522,5.0562,...,8.4086,5208.3708,5083.5146,5180.9846,8.5763,8.4147,44213866.9939,5162.1341,69764317.2988,9.8252


In [56]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split


def lasso_regression(train_df, target_column, test_size=0.1, random_state=42):
    # split data into features and target
    X = train_df.drop(columns=[target_column])
    y = train_df[target_column]
    
    # split into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    print("Performing Lasso regression...")
    
    # Lasso regression with cross-validation for hyperparameter tuning of regularization parameter alpha
    lasso = LassoCV(cv=5, random_state=random_state, max_iter=10000).fit(X_train, y_train)
    
    selected_features_lasso = np.where(lasso.coef_ != 0)[0]
    selected_feature_names_lasso = X.columns[selected_features_lasso]

    print(f"Selected features by Lasso ({len(selected_feature_names_lasso)}): {selected_feature_names_lasso}")

    print("Predicting on the training and test data (Lasso)...")

    y_train_pred_lasso = lasso.predict(X_train)
    y_test_pred_lasso = lasso.predict(X_test)

    print("Evaluating the model (Lasso)...")
    
    # evaluating on train data
    train_rmse_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
    train_mae_lasso = mean_absolute_error(y_train, y_train_pred_lasso)
    train_mape_lasso = mean_absolute_percentage_error(y_train, y_train_pred_lasso)

    # evaluating on test data
    test_rmse_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
    test_mae_lasso = mean_absolute_error(y_test, y_test_pred_lasso)
    test_mape_lasso = mean_absolute_percentage_error(y_test, y_test_pred_lasso)

    # collect metrics
    results_lasso_dict = {
        "Model": ["Lasso"] * 6,
        "Metric": ["Training RMSE", "Training MAE", "Training MAPE",
                   "Testing RMSE", "Testing MAE", "Testing MAPE"],
        "Value": [f"{train_rmse_lasso:.2f}", f"{train_mae_lasso:.2f}", f"{train_mape_lasso:.2f}",
                  f"{test_rmse_lasso:.2f}", f"{test_mae_lasso:.2f}", f"{test_mape_lasso:.2f}"]
    }

    results_lasso_df = pd.DataFrame(results_lasso_dict)
    print(results_lasso_df)
    
    # collect variable importance
    variable_importance_dict = {
        "Feature": selected_feature_names_lasso,
        "Coefficient": lasso.coef_[selected_features_lasso]
    }
    variable_importance_df = pd.DataFrame(variable_importance_dict)
    variable_importance_df = variable_importance_df.sort_values(by="Coefficient", ascending=False).reset_index(drop=True)

    print("\nVariable Importance (Lasso):")
    print(variable_importance_df)
    
    #return results_lasso_df, variable_importance_df


In [58]:
lasso_regression(df, 'cTOTEXn', random_state = 45)

Performing Lasso regression...
Selected features by Lasso (4): Index(['rPeakload.injection.per.Peakload.N6_poly',
       'rMeters.active.share.all_poly', 'rUsageHours.delivered.N5_poly',
       'rUsageHours.delivered.N7_poly'],
      dtype='object')
Predicting on the training and test data (Lasso)...
Evaluating the model (Lasso)...
   Model         Metric         Value
0  Lasso  Training RMSE  143361603.02
1  Lasso   Training MAE   64500609.36
2  Lasso  Training MAPE          3.31
3  Lasso   Testing RMSE   66808362.19
4  Lasso    Testing MAE   48208302.56
5  Lasso   Testing MAPE          3.14

Variable Importance (Lasso):
                                    Feature  Coefficient
0  rPeakload.injection.per.Peakload.N6_poly       0.7551
1             rMeters.active.share.all_poly       0.7357
2             rUsageHours.delivered.N5_poly       0.7301
3             rUsageHours.delivered.N7_poly       0.3364


In [48]:
lasso_regression(df, 'cTOTEXn', random_state = 45)

Performing Lasso regression...
Selected features by Lasso (5): Index(['yInjection.generation.tot', 'yInjection.dec.sum',
       'yEnergy.delivered.N1357.sum', 'yEnergy.delivered.net.N2to4.sum',
       'yEnergy.delivered.net.N5to7.sum'],
      dtype='object')
Predicting on the training and test data (Lasso)...
Evaluating the model (Lasso)...
   Model         Metric        Value
0  Lasso  Training RMSE  13979682.62
1  Lasso   Training MAE   6867632.27
2  Lasso  Training MAPE         0.21
3  Lasso   Testing RMSE  53658764.30
4  Lasso    Testing MAE  17172598.66
5  Lasso   Testing MAPE         0.21

Variable Importance (Lasso):
                           Feature  Coefficient
0      yEnergy.delivered.N1357.sum       0.0099
1  yEnergy.delivered.net.N5to7.sum       0.0078
2        yInjection.generation.tot       0.0028
3               yInjection.dec.sum       0.0023
4  yEnergy.delivered.net.N2to4.sum      -0.0027
